In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
df = pd.read_csv('wfp_market_food_prices.zip', encoding='ISO-8859-1')

## Data Cleaning

In [3]:
# Rename columns to something sensible
df = df.rename(
    {
        'adm0_id': 'country_id',
        'adm0_name': 'country_name',
        'adm1_id': 'locality_id',
        'adm1_name': 'locality_name',
        'mkt_id': 'market_id',
        'mkt_name': 'market_name',
        'cm_id': 'commodity_id',
        'cm_name': 'commodity_name',
        'cur_id': 'currency_id',
        'cur_name': 'currency_name',
        'pt_id': 'market_type_id',
        'pt_name': 'market_type',
        'um_id': 'measurement_id',
        'um_name': 'unit_of_goods_measurement',
        'mp_price': 'price',
        'mp_month': 'month_recorded',
        'mp_year': 'year_recorded',
        'mp_commoditysource': 'source'
    },
    axis='columns')

useless_columns = ['country_id', 'locality_id', 'market_id', 'commodity_id',
                   'currency_id', 'market_type_id', 'measurement_id', 'source']
df = df.drop(useless_columns, axis=1)

In [4]:
# For a given country, locality, market and commodity, only keep the most recent recording of price
# Since the data is sorted to begin with, we only need to keep the last entry
df = df.groupby(['country_name', 'locality_name', 'market_name', 'commodity_name']).last().reset_index()
df = df.drop(['month_recorded', 'year_recorded'], axis=1)

In [5]:
# Remove all parentheses, which merely differentiate between the different kinds of food
# i.e. imported or not, method of cooking, etc.
df.commodity_name = df.commodity_name.apply(lambda s: re.sub('\(.*\)', '', s).strip())

In [6]:
# Remove things that are not foods
non_foods = ['Exchange rate', 'Fuel', 'Wage', 'Charcoal', 'Oil', 'Transport', 'Cotton']
df = df[~df.loc[:, 'commodity_name'].isin(non_foods)]

In [7]:
# Remove duplicates
df = df.drop_duplicates(subset=['country_name', 'locality_name', 'market_name', 'commodity_name'])

In [8]:
# Renumber rows
df = df.reset_index(drop=True)

In [9]:
df.to_csv('pricing.csv')

In [10]:
# FIXME there are some commodities that aren't bought by the kg...
# but our nutrition data is per kg!
df[df.unit_of_goods_measurement != 'KG'].commodity_name.unique()

array(['Livestock', 'Bread', 'Cheese', 'Eggs', 'Fish', 'Milk', 'Pasta',
       'Tea', 'Potatoes', 'Noodles', 'Rice', 'Sugar', 'Beans',
       'Groundnuts', 'Maize', 'Onions', 'Sorghum', 'Cassava', 'Peas',
       'Sesame', 'Soybeans', 'Cocoa', 'Coffee', 'Wheat', 'Millet',
       'Apples', 'Bananas', 'Cabbage', 'Carrots', 'Cashew', 'Fonio',
       'Garlic', 'Maize flour', 'Meat', 'Oranges', 'Salt',
       'Sweet potatoes', 'Tomatoes', 'Wheat flour', 'Plantains', 'Yam',
       'Maize meal', 'Lettuce', 'Cowpeas', 'Gari', 'Avocados', 'Labaneh',
       'Water', 'Yogurt', 'Parsley', 'Sour cream'], dtype=object)

## Adding Nutrition Data

All lovingly added by hand from [the USDA databases](https://ndb.nal.usda.gov/ndb/search/list).

In [11]:
# Protein (g), Fat (g), Carb. (g), Fiber (g)

nutrition_dict = {
    'Bread': [10.67, 4.53, 47.54, 4.0],  # Wheat bread
    'Rice': [3.99, 0.34, 21.34, 1.8],  # Wild rice, cookied
    'Wheat': [7.49, 1.27, 42.53, 1.1],  # Wheat, sprouted
    'Livestock': [17.44, 8.10, 0.04, 0.0],  # Chicken
    'Apples': [0.93, 0.32, 65.89, 8.7],  # Apples, dried, sulfured, uncooked 
    'Bananas': [3.89, 1.81, 88.28, 9.9],  # Bananas, dehydrated, or banana powder
    'Beans': [1.80, 3.20, 2.80, 0.1],  # Beans, liquid from stewed kidney beans
    'Carrots': [0.93, 0.24, 9.58, 2.8],  # Carrots, raw 
    'Cheese': [26.96, 30.99, 1.44, 0.0],  # Cheese, swiss 
    'Eggs': [48.05, 43.90, 1.13, 0.0],  # Egg, whole, dried 
    'Fish': [20.42, 0.25, 0.00, 0.0],  # Fish, cod, Pacific, cooked 
    'Lentils': [24.63, 1.06, 63.35, 10.7],  # Lentils, raw
    'Meat': [18.17, 18.98, 0.47, 0.0],  # Beef, cured, corned beef, brisket, cooked 
    'Milk': [45.71, 17.14, 18.50, 7.1], # Protein supplement, milk based, Muscle Milk, powder
    'Onions': [1.10, 0.10, 9.34, 1.7],  # Onions, raw 
    'Pasta': [14.29, 1.79, 73.21, 8.9],  # SEVERINO, HOMEMADE PASTA, WAGON WHEELS PASTA, UPC: 029737022239 
    'Potatoes': [2.57, 0.10, 12.44, 2.5],  # Potatoes, raw, skin 
    'Sugar': [0.12, 0.00, 98.09, 0.0],  # Sugars, brown 
    'Tea': [0, 0, 20, 20],  # TEAS'TEA, ORGANIC LIGHTLY SWEET HIBISCUS GREEN TEA, UPC: 835143011305. Per bottle (500ml)
    'Tomatoes': [0.79, 0.25, 3.47, 1.9],  # Tomatoes, red, ripe, canned, packed in tomato juice 
    'Cabbage': [1.28, 0.10, 5.80, 2.5],  # Cabbage, raw
    'Cucumbers': [0.59, 0.16, 2.16, 0.7],  # Cucumber, peeled, raw
    'Wheat flour': [11.98, 1.66, 72.53, 2.4],  # Wheat flours, bread, unenriched
    'Cassava meal': [1.36, 0.28, 38.06, 1.8], # Cassava, raw 
    'Maize': [8.75, 5.09, 73.89, 8.4],  # Corn flour, whole-grain, blue (harina de maiz morado) 
    'Sorghum': [10.62, 3.46, 72.09, 6.7],  # Sorghum grain 
    'Millet': [3.51, 1.00, 23.67, 1.3],  # Millet, cooked
    'Noodles': [1.79, 0.20, 24.01, 1.0],  # Rice noodles, cooked
    'Cassava flour': [1.36, 0.28, 38.06, 1.8], # Cassava, raw 
    'Sweet potatoes': [1.98, 0.20, 23.19, 1.7], # Sweet potato, canned, mashed
    'Groundnuts': [18.22, 43.85, 43.85, 3.3], # Nuts, cashew nuts, raw 
    'Cassava': [1.36, 0.28, 38.06, 1.8], # Cassava, raw 
    'Peas': [2.80, 0.20, 7.55, 2.6], # Peas, edible-podded, raw
    'Sesame': [40.32, 11.89, 35.14, 0.0], # Seeds, sesame flour, partially defatted 
    'Soybeans': [12.95, 6.80, 11.05, 4.2],  # Soybeans, green, raw
    'Blackberry': [1.39, 0.49, 9.61, 5.3],  # Blackberries, raw 
    'Broccoli': [3.83, 0.52, 3.12, 2.8],  # Broccoli raab, cooked 
    'Cauliflower': [3.04, 0.31, 6.28, 3.3],  # Cauliflower, green, cooked, with salt 
    'Chickpeas': [22.39, 6.69, 57.82, 10.8],  # Chickpea flour (besan)
    'Cocoa': [19.60, 13.70, 57.90, 37.0],  # Cocoa, dry powder, unsweetened 
    'Coffee': [0.09, 0.0, 0.75, 0.0],  # Beverages, coffee, instant, chicory 
    'Mangoes': [0.82, 0.38, 14.98, 1.6],  # Mangos, raw
    'Oranges': [1.30, 0.30, 15.50, 4.5],  # Oranges, raw, with peel 
    'Papaya': [0.47, 0.26, 10.82, 1.7],  # Papayas, raw
    'Plantains': [1.52, 0.16, 41.37, 2.2],  # Plantains, yellow, baked 
    'Pumpkin': [1.00, 0.10, 6.50, 0.5],  # Pumpkin, raw
    'Spinach': [2.86, 0.39, 3.63, 2.2],  # Spinach, raw
    'Tamarillos/tree tomatoes': [0.79, 0.25, 3.47, 1.9],  # Tomatoes, red, ripe, canned, packed in tomato juice 
    'Garlic': [6.36, 0.50, 33.06, 2.1],  # Garlic, raw 
    'Guava': [2.55, 0.95, 14.32, 5.4],  # Guavas, common, raw
    'Maize flour': [8.75, 5.09, 73.89, 8.4],  # Corn flour, whole-grain, blue (harina de maiz morado) 
    'Salt': [0.0, 0.0, 0.0, 0.0],  # Salt, table 
    'Cornstarch': [0.26, 0.05, 91.27, 0.9],  # Cornstarch 
    'Peanut': [25.80, 49.24, 16.13, 8.5],  # Peanuts, all types, raw 
    'Yam': [1.53, 0.17, 27.88, 4.1],  # Yam, raw 
    'Cashew': [5.00, 14.00, 8.00, 1.0],  # CASHEWS, UPC: 099482434236 
    'Maize meal': [8.75, 5.09, 73.89, 8.4],  # Corn flour, whole-grain, blue (harina de maiz morado) 
    'Fonio': [3.51, 1.00, 23.67, 1.3],  # Millet, cooked
    'Ghee': [0.0, 100.0, 0.0, 0.0],  # Butter, Clarified butter (ghee)
    'Bulgur': [3.08, 0.24, 18.58, 4.5],  # Bulgur, cooked 
    'Lettuce': [1.36, 0.15, 2.87, 1.3],  # Lettuce, green leaf, raw 
    'Cowpeas': [4.67, 0.10, 2.80, 0.0],  # Cowpeas, leafy tips, cooked, boiled, drained, with salt 
    'Pulses': [25.80, 49.24, 16.13, 8.5],  # Peanuts, all types, raw 
    'Gari': [1.79, 1.79, 78.57, 0.0],  # GARI, UPC: 611785102454 
    'Poultry': [17.44, 8.10, 0.04, 0.0],  # Chicken
    'Avocados': [1.96, 15.41, 8.64, 6.8],  # Avocados, raw, California
    'Chili': [0.72, 0.27, 4.60, 1.7],  # Peppers, chili, green, canned 
    'Eggplants': [0.98, 0.18, 5.88, 3.0],  # Eggplant, raw 
    'Passion fruit': [2.20, 0.70, 23.38, 10.4],  # Passion-fruit, (granadilla), purple, raw 
    'Peppers': [0.72, 0.27, 4.60, 1.7],  # Peppers, chili, green, canned 
    'Sorghum flour': [10.62, 3.46, 72.09, 6.7],  # Sorghum grain 
    'Cassava leaves': [1.36, 0.28, 38.06, 1.8], # Cassava, raw 
    'Zucchini': [1.21, 0.32, 3.11, 1.0],  # Squash, summer, zucchini, includes skin, raw
    'Labaneh': [7.49, 6.17, 5.73, 0.0],  # YOGHURT, UPC: 853923002800 
    'Water': [0.0, 0.0, 0.0, 0.0],  # Water, bottled, generic 
    'Yogurt': [7.49, 6.17, 5.73, 0.0],  # YOGHURT, UPC: 853923002800 
    'Dates': [1.81, 0.15, 74.97, 6.7],  # Dates, medjool 
    'Parsley': [2.97, 0.79, 6.33, 3.3],  # Parsley, fresh 
    'Beetroots': [1.00, 0.00, 13.00, 2.00],  # MASHED BEETROOT, UPC: 5902009001581 
    'Buckwheat grits': [13.25, 3.40, 71.50, 10.0],  # Buckwheat
    'Butter': [0.0, 100.0, 0.0, 0.0],  # Butter, Clarified butter (ghee)
    'Curd': [12.50, 8.10, 6.90, 0.0],  # Soybean, curd cheese 
    'Fat': [0.0, 100.0, 0.0, 0.0],  # Butter, Clarified butter (ghee)
    'Sour cream': [2.44, 19.35, 4.63, 0.0],  # Cream, sour, cultured 
}

In [12]:
# USDA data is per 10g, multiply by 10 to be per kg!
nutrition = 10 * pd.DataFrame(nutrition_dict,
                              index=['Protein', 'Fat', 'Carb', 'Fiber']).transpose()

In [13]:
nutrition.to_csv('nutrition.csv')